In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cyclistic Bike-Share Case Study

# **Introduction**

This project is a case study of a fictional company, Cyclistic, a bike-share program based in Chicago that features more than 5,800 bicycles and 600 docking stations. The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders.From the insights gotten, a new marketing strategy will be designed to convert casual riders into annual members.
As a junior data analyst for Cyclistic, I was tasked to make data-driven recommendations for the marketing campaign.
I will complete this analysis using the six phases of the data analysis process: ask, prepare, process, analyze, and act

# Prepare Phase

In [ ]:
#Load required packages
library(tidyverse)
library(janitor)
library(lubridate)
library(skimr)
library(data.table)

In [ ]:
#Load data 
Apr20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Apr20.csv')
May20 <- read.csv('../input/google-capstone-project-case-study/Case Study/May20.csv')
Jun20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Jun20.csv')
Jul20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Jul20.csv')
Aug20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Aug20.csv')
Sep20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Sep20.csv')
Oct20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Oct20.csv')
Nov20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Nov20.csv')
Dec20 <- read.csv('../input/google-capstone-project-case-study/Case Study/Dec20.csv')
Jan21 <- read.csv('../input/google-capstone-project-case-study/Case Study/Jan21.csv')
Feb21 <- read.csv('../input/google-capstone-project-case-study/Case Study/Feb21.csv')
Mar21 <- read.csv('../input/google-capstone-project-case-study/Case Study/Mar21.csv')

In [ ]:
#Looking at the names of the columns
colnames(Apr20)
colnames(May20)
colnames(Jun20)
colnames(Jul20)
colnames(Aug20)
colnames(Sep20)
colnames(Oct20)
colnames(Nov20)
colnames(Dec20)
colnames(Jan21)
colnames(Feb21)
colnames(Mar21)

In [ ]:
str(Apr20)
str(May20)
str(Jun20)
str(Jul20)
str(Aug20)
str(Sep20)
str(Oct20)
str(Nov20)
str(Dec20)
str(Jan21)
str(Feb21)
str(Mar21)

Columns start_station_id and end_station_id from December 2020 - March 2021 have char as data type. Whereas data from April 2020 - November 2020 have integer as data type for the same columns. To fix this, I will convert data from April 2020 - November 2020 to char.

In [ ]:
Dec20 <-  mutate(Dec20, start_station_id = as.double(start_station_id),
                 end_station_id = as.double(end_station_id))
Jan21 <-  mutate(Jan21, start_station_id = as.double(start_station_id),
                 end_station_id = as.double(end_station_id))
Feb21 <-  mutate(Feb21, start_station_id = as.double(start_station_id),
                 end_station_id = as.double(end_station_id))
Mar21 <-  mutate(Mar21, start_station_id = as.double(start_station_id),
                 end_station_id = as.double(end_station_id))

In [ ]:
#Joining all the datasets into one
all_trip <- 
   rbind(Apr20,May20, Jun20, Jul20, Aug20, Sep20, Oct20, Nov20, Dec20, Jan21, Feb21, Mar21)

In [ ]:
#Renaming columns for easy interpretation
rename(all_trip, ride_type = rideable_type, start_time = started_at, end_time = ended_at, customer_type = member_casual)
str(all_trip)

In [ ]:
dim(all_trip)

In [ ]:
all_trip <- rename(all_trip, ride_type = rideable_type, start_time = started_at, end_time = ended_at, customer_type = member_casual)
str(all_trip)

In [ ]:
write.csv(all_trip, file = "all_trips.csv", row.names = FALSE)

In [ ]:
#Removing unwanted columns for this analysis
all_trip <- all_trip %>%
select(-(start_lat:end_lng))
str(all_trip)

In [ ]:
# Convert started_at and ended_at columns to datetime data type
all_trip[['start_time']] <- ymd_hms(all_trip[['start_time']])
all_trip[['end_time']] <- ymd_hms(all_trip[['end_time']])

In [ ]:
all_trip$time <- format(all_trip$start_time, format = "%H:%M")
all_trip$time <- as.POSIXct(all_trip$time, format = "%H:%M")

In [ ]:
# ride_length column in minutes
all_trip <- all_trip %>% mutate(ride_length = signif(difftime(end_time, start_time, units = 'mins'),2))

# day_of_week column
all_trip <- all_trip %>% mutate(day_of_week = weekdays(start_time))

# month column
all_trip <- all_trip %>% mutate(all_trip, date = as.Date(all_trip$start_time)) %>%
    mutate(month = format(as.Date(date), "%B"))

In [ ]:
#Checking for ride lengths less than a minute and greater than 24hrs
nrow(subset(all_trip,ride_length < 0))
nrow(subset(all_trip,ride_length > 1440))

#checking for testrides that were made by company for quality checks
nrow(subset(all_trip, start_station_name %like% "TEST"))
nrow(subset(all_trip, start_station_name %like% "test"))
nrow(subset(all_trip, start_station_name %like% "Test"))

In [ ]:
#Creating dataframes without Negative ride lengths
all_trip_2 <- all_trip[!(all_trip$ride_length <= 0),]
  
#Creating dataframes without Test rides
all_trip_2 <- all_trip_2[!((all_trip_2$start_station_name %like% "TEST" | all_trip_2$start_station_name %like% "test")),]

In [ ]:
head(all_trip_2)

In [ ]:
dim(all_trip_2)

In [ ]:
#Checking for duplicate rows
all_trip_2 %>% distinct()

In [ ]:
#checking for 2 distinct categories in the customer segment
table(all_trip_2$customer_type)

# Analyze Phase

In [ ]:
summary(all_trip_2$ride_length)

In [ ]:
all_trip_2 %>%
    summarise(min_ride_length = min(ride_length), 
              max_ride_length = max(ride_length),
              mean_ride_length = mean(ride_length))

In [ ]:
#aggregating total trip duration by customer type
setNames(aggregate(ride_length ~ customer_type, all_trip_2, sum), c("customer_type", "total_trip_duration(mins)"))

In [ ]:
#Creating the average trip duration by customer type
aggregate(all_trip_2$ride_length ~ all_trip_2$customer_type,FUN=mean)

In [ ]:
#statistical summary of trip_duration by customer_type
all_trip_2 %>%
    group_by(customer_type) %>%
    summarise(min_trip_duration = min(ride_length),max_trip_duration = max(ride_length),
              median_trip_duration = median(ride_length), mean_trip_duration = mean(ride_length))

In [ ]:
#Assign the correct order to each day of the week
all_trip_2$day_of_week <- 
    ordered(all_trip_2$day_of_week, levels = c('Monday', 'Tuesday', 'Wednesday', 
                                                    'Thursday', 'Friday', 'Saturday', 'Sunday'))

In [ ]:
#Average ride length between casual and member riders for each day of the week
aggregate(all_trip_2$ride_length ~ all_trip_2$customer_type + all_trip_2$day_of_week,FUN=mean)

In [ ]:
#Number of ride between casual and member riders for each day of the week
all_trip_2 %>%
    group_by(customer_type, day_of_week) %>%
    summarise(number_of_ride = n(), .groups = 'drop') %>%
    arrange(day_of_week)

In [ ]:
#Assigning the correct order to each month of the year
all_trip_2$month <-
    ordered(all_trip_2$month, levels = c('January', 'February', 'March', 'April', 'May', 'June', 'July',
                                              'August', 'September', 'October', 'November', 'December'))

In [ ]:
#Number of ride between casual and member riders per month of the year
all_trip_2 %>%
    group_by(customer_type, month) %>%
    summarise(number_of_ride = n(), .groups = 'drop') %>%
    arrange(month)

In [ ]:
#Average ride length between casual and member riders per month of the year
all_trip_2 %>%
    group_by(customer_type, month) %>%
    summarise(average_ride_length = mean(ride_length), .groups = 'drop') %>%
    arrange(month)

In [ ]:
#Comparing bike type preferences between casual and member riders
all_trip_2 %>%
    group_by(ride_type, customer_type) %>%
    summarise(number_of_ride = n(), .groups = 'drop')

# Share Phase

This phase includes visualizations that can be presented to the stakeholders to better understand the data and make informed  decisions as well as adequate maerketing strategies.

In [ ]:
#Visualize average ride length by casual member for each day of the week
all_trip_2 %>%  
  group_by(customer_type, day_of_week) %>% 
  summarise(number_of_rides = n()) %>% 
  arrange(customer_type, day_of_week)  %>% 
  ggplot(aes(x = day_of_week, y = number_of_rides, fill = customer_type)) +
  labs(title ="Total trips by customer type Vs. Day of the week") +
  geom_col(width=0.5, position = position_dodge(width=0.5)) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))

In [ ]:
#Visualize average ride length by customer types for each day of the week
all_trip_2 %>%
    group_by(customer_type, day_of_week) %>%
    summarise(average_ride_duration = mean(ride_length), .groups = 'drop') %>%
    ggplot(aes(x = day_of_week, y = average_ride_duration, fill = customer_type)) + 
    geom_bar(position = "dodge", stat = "identity")+
    geom_col(width=0.5, position = position_dodge(width=0.5))

In [ ]:
#Visualize total trips by customer type for each month of the year
all_trip_2 %>%  
  group_by(customer_type, month) %>% 
  summarise(number_of_rides = n()) %>% 
  arrange(customer_type, month)  %>% 
  ggplot(aes(x = month, y = number_of_rides, fill = customer_type)) +
  labs(title ="Total trips by customer type Vs. Month") +
  theme(axis.text.x = element_text(angle = 30)) +
  geom_col(width=0.5, position = position_dodge(width=0.5)) +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))

In [ ]:
#Visualize casual rider's number of ride for each month 
all_trip_2 %>%
    group_by(month, customer_type) %>%
    summarise(number_of_ride = n(), .groups = 'drop') %>%
    filter(customer_type == 'casual') %>%
    ggplot(aes(x = month, y = number_of_ride, fill = customer_type)) +
    geom_bar(position = 'dodge', stat = 'identity') + 
    theme(axis.text.x = element_text(angle = 45))

In [ ]:
#Visualizaton of average trip duration by customer type Vs. month
all_trip_2 %>%  
  group_by(customer_type, month) %>% 
  summarise(average_trip_duration = mean(ride_length)) %>%
  ggplot(aes(x = month, y = average_trip_duration, fill = customer_type)) +
  geom_col(width=0.5, position = position_dodge(width=0.5)) + 
  labs(title ="Average trip duration by customer type Vs. Month") +
  theme(axis.text.x = element_text(angle = 30))

In [ ]:
#Visualization of the Hourly trends 
all_trip_2 %>%  
  group_by(customer_type, time) %>% 
  summarise(number_of_trips = n()) %>%
  ggplot(aes(x = time, y = number_of_trips, color = customer_type, group = customer_type)) +
  geom_line() +
  scale_x_datetime(date_breaks = "1 hour", minor_breaks = NULL,
                   date_labels = "%H:%M", expand = c(0,0)) +
  theme(axis.text.x = element_text(angle = 90)) +
  labs(title ="Demand over 24 hours of a day", x = "Time of the day")